In [ ]:
import Data_Conversion
import Input_Parameters 
import Data_Conversion
import Passive_Model
import Solar_Generation
import Electrical_Load
import Baseline_CO

In [2]:
import numpy as np
import pandas as pd

In [ ]:
# Sensitivity Analysis on Locations, Linear Programming
# Fred Fan, Stanford University, 03/17/2025

In [ ]:
# Define the base data directory, list of locations, and the weather year.
data_dir = "Data"
locations = ["Arizona", "Alaska", "Minnesota", "Florida", "HalfMoonBay"]

# 20 years of training data
weather_year_list_training = list(range(1998, 1999))  # use list(range(1998, 2018)) for full dataset
# 5 years of training data
weather_year_list_testing = list(range(2022, 2023))  # use list(range(2018, 2023)) for full dataset

# Set a random seed for consistency
# Define the number of rows (i) and columns (j)
i = len(locations)  # Number of rows
j = len(weather_year_list_training) + len(weather_year_list_testing)  # Number of columns

# Generate a sequential array of numbers starting from 1
total_cells = i * j  # Total number of cells in the DataFrame
sequential_numbers = np.arange(1, total_cells + 1)  # Generates 1, 2, 3, ..., i*j

# Reshape the array into a 2D array with i rows and j columns
data = sequential_numbers.reshape(i, j)

# Convert the array into a DataFrame
random_seeds = pd.DataFrame(data, columns=[f'Column_{k+1}' for k in range(j)])

lats, lons, timezones = Data_Conversion.get_timezones(data_dir, locations)

Reading file for Arizona: Data\NREL_NSRDB_Arizona\323705_33.45_-112.06_1998.csv
Reading file for Alaska: Data\NREL_NSRDB_Alaska\34776_59.25_-154.62_1998.csv
Reading file for Minnesota: Data\NREL_NSRDB_Minnesota\777900_44.97_-93.26_1998.csv
Reading file for Florida: Data\NREL_NSRDB_Florida\1060499_25.77_-80.18_1998.csv
Reading file for HalfMoonBay: Data\NREL_NSRDB_HalfMoonBay\137344_37.49_-122.42_1998.csv


In [4]:
# Create the nested dictionary
training_results = {location: {year: {} for year in weather_year_list_training} for location in locations}

In [ ]:
# Sensitivity Analysis on Locations
for i in range(len(locations)):
    for j in range(len(weather_year_list_training)):
        
        location = locations[i]
        year = weather_year_list_training[j]

        # Get a unique random seed number
        random_seed = random_seeds.iloc[i, j]
        print(random_seed)
        
        # Read NSRDB weather data of the given location of the given year
        NSRDB_raw_weather = Data_Conversion.read_NSRDB(data_dir, location, year)

        # Prepare weather data file using NSRDB data
        weather_data = Data_Conversion.prepare_NSRDB(NSRDB_raw_weather, lats[i], lons[i], timezones[i])

        # Prepare heating and cooling load using weather data and passive model
        NetHeatTransfers = Passive_Model.passive_model(Input_Parameters.calibration_file_path, weather_data, Input_Parameters.T_indoor_constant, lats[i])

        # Prepare solar PV capacity factor using weather data
        pv_cf = Solar_Generation.generate_pv(weather_data, lats[i])

        # Prepare occupancy and electrical load schedule using for a specific random seed number for a specific year at a specific location
        load_sched = Electrical_Load.generate_schedules("bayes", weather_data, random_seed)
        
        # Combine all relative input data as input_df, which will be the input of the capacity optimization algorithm
        input_df = Data_Conversion.combine_input_NSRDB(weather_data, load_sched, pv_cf, NetHeatTransfers)

        # Run optimization function
        PV_Size, Battery_Size, PCM_Heating_Size, PCM_Cooling_Size, ObjValue, First_stage_cost, Second_stage_cost = Baseline_CO.Cap_Baseline_V1(input_df, Input_Parameters.lossofloadcost)

        # Store the variables in the nested dictionary
        training_results[location][year] = {
            'PV_Size': PV_Size,
            'Battery_Size': Battery_Size,
            'PCM_Heating_Size': PCM_Heating_Size,
            'PCM_Cooling_Size': PCM_Cooling_Size,
            'Total Cost': ObjValue,
            'Capital Cost': First_stage_cost,
            'Operation Cost': Second_stage_cost
        }



2
Reading file for Arizona: Data\NREL_NSRDB_Arizona\323705_33.45_-112.06_1998.csv
Read LP format model from file C:\Users\Fred\AppData\Local\Temp\tmptkpvha7i.pyomo.lp
Reading time = 0.39 seconds
x1: 157682 rows, 140165 columns, 424810 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 157682 rows, 140165 columns and 424810 nonzeros
Model fingerprint: 0x5bc2e45c
Coefficient statistics:
  Matrix range     [2e-03, 6e+00]
  Objective range  [3e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-12, 2e+01]
Presolve removed 54016 rows and 42513 columns
Presolve time: 0.37s
Presolved: 103666 rows, 97652 columns, 426770 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.11s

Barrier

In [6]:
print(training_results)

{'Arizona': {1998: {'PV_Size': 8.812, 'Battery_Size': 16.181, 'PCM_Heating_Size': 89.156, 'PCM_Cooling_Size': 0.0, 'Total Cost': 22717.393, 'Capital Cost': 22717.393, 'Operation Cost': 0.0}}, 'Alaska': {1998: {'PV_Size': 207.619, 'Battery_Size': 29.575, 'PCM_Heating_Size': 212.496, 'PCM_Cooling_Size': 0.0, 'Total Cost': 42777.755, 'Capital Cost': 42777.755, 'Operation Cost': 0.0}}, 'Minnesota': {1998: {'PV_Size': 43.902, 'Battery_Size': 8.889, 'PCM_Heating_Size': 164.988, 'PCM_Cooling_Size': 0.0, 'Total Cost': 26604.113, 'Capital Cost': 26604.113, 'Operation Cost': 0.0}}, 'Florida': {1998: {'PV_Size': 7.343, 'Battery_Size': 5.111, 'PCM_Heating_Size': 0.0, 'PCM_Cooling_Size': 48.202, 'Total Cost': 20307.367, 'Capital Cost': 20307.367, 'Operation Cost': 0.0}}, 'HalfMoonBay': {1998: {'PV_Size': 9.599, 'Battery_Size': 1.628, 'PCM_Heating_Size': 69.454, 'PCM_Cooling_Size': 0.0, 'Total Cost': 20652.282, 'Capital Cost': 20652.282, 'Operation Cost': 0.0}}}


In [7]:
# Initialize a dictionary to store averaged results
training_averaged_results = {}

# Iterate over each location in the original dictionary (results)
for location, years_data in training_results.items():
    # Initialize a dictionary to store sums and counts for averaging
    sums = {
        'PV_Size': 0,
        'Battery_Size': 0,
        'PCM_Heating_Size': 0,
        'PCM_Cooling_Size': 0,
        'Total Cost': 0,
        'Capital Cost': 0,
        'Operation Cost': 0
    }
    count = 0

    # Iterate over each year for the location
    for year, data in years_data.items():
        for key in sums:
            sums[key] += data[key]
        count += 1

    # Calculate averages and store in training_averaged_results
    training_averaged_results[location] = {key: value / count for key, value in sums.items()}

# Print the averaged results
print(training_averaged_results)

{'Arizona': {'PV_Size': 8.812, 'Battery_Size': 16.181, 'PCM_Heating_Size': 89.156, 'PCM_Cooling_Size': 0.0, 'Total Cost': 22717.393, 'Capital Cost': 22717.393, 'Operation Cost': 0.0}, 'Alaska': {'PV_Size': 207.619, 'Battery_Size': 29.575, 'PCM_Heating_Size': 212.496, 'PCM_Cooling_Size': 0.0, 'Total Cost': 42777.755, 'Capital Cost': 42777.755, 'Operation Cost': 0.0}, 'Minnesota': {'PV_Size': 43.902, 'Battery_Size': 8.889, 'PCM_Heating_Size': 164.988, 'PCM_Cooling_Size': 0.0, 'Total Cost': 26604.113, 'Capital Cost': 26604.113, 'Operation Cost': 0.0}, 'Florida': {'PV_Size': 7.343, 'Battery_Size': 5.111, 'PCM_Heating_Size': 0.0, 'PCM_Cooling_Size': 48.202, 'Total Cost': 20307.367, 'Capital Cost': 20307.367, 'Operation Cost': 0.0}, 'HalfMoonBay': {'PV_Size': 9.599, 'Battery_Size': 1.628, 'PCM_Heating_Size': 69.454, 'PCM_Cooling_Size': 0.0, 'Total Cost': 20652.282, 'Capital Cost': 20652.282, 'Operation Cost': 0.0}}


In [ ]:
# Testing:

# Create the nested dictionary
testing_results = {location: {year: {} for year in weather_year_list_testing} for location in locations}

for i in range(len(locations)):
    location = locations[i]
    test_capacities = [training_averaged_results[location]['PV_Size'], training_averaged_results[location]['Battery_Size'], training_averaged_results[location]['PCM_Heating_Size'], training_averaged_results[location]['PCM_Cooling_Size']]
    for j in range(len(weather_year_list_testing)):
        year = weather_year_list_testing[j]
        
        # Get a unique random seed number
        random_seed = random_seeds.iloc[i, j+len(weather_year_list_training)]
        print(random_seed)
        
        # Read NSRDB weather data of the given location of the given year
        NSRDB_raw_weather = Data_Conversion.read_NSRDB(data_dir, location, year)

        # Prepare weather data file using NSRDB data
        weather_data = Data_Conversion.prepare_NSRDB(NSRDB_raw_weather, lats[i], lons[i], timezones[i])

        # Prepare heating and cooling load using weather data and passive model
        NetHeatTransfers = Passive_Model.passive_model(Input_Parameters.calibration_file_path, weather_data, Input_Parameters.T_indoor_constant, lats[i])

        # Prepare solar PV capacity factor using weather data
        pv_cf = Solar_Generation.generate_pv(weather_data, lats[i])

        # Prepare occupancy and electrical load schedule using for a specific random seed number for a specific year at a specific location
        load_sched = Electrical_Load.generate_schedules("bayes", weather_data, random_seed)
        
        # Combine all relative input data as input_df, which will be the input of the capacity optimization algorithm
        input_df = Data_Conversion.combine_input_NSRDB(weather_data, load_sched, pv_cf, NetHeatTransfers)
    
        ObjValue, First_stage_cost, Second_stage_cost = Baseline_CO.simulate(input_df, Input_Parameters.lossofloadcost, test_capacities)
        # Store the variables in the nested dictionary
        testing_results[location][year] = {
            'Total Cost': ObjValue,
            'Capital Cost': First_stage_cost,
            'Operation Cost': Second_stage_cost
        }

3
Reading file for Arizona: Data\NREL_NSRDB_Arizona\323705_33.45_-112.06_2022.csv
Read LP format model from file C:\Users\Fred\AppData\Local\Temp\tmp5l5qvljk.pyomo.lp
Reading time = 0.40 seconds
x1: 157682 rows, 140160 columns, 385430 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 157682 rows, 140160 columns and 385430 nonzeros
Model fingerprint: 0xd7a2da2f
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+03, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-14, 9e+01]
Presolve removed 95144 rows and 56087 columns
Presolve time: 0.21s
Presolved: 62538 rows, 84073 columns, 195808 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.04s

Barrier 

In [9]:
# Initialize a dictionary to store averaged results
testing_averaged_results = {}

# Iterate over each location
for location, years_data in testing_results.items():
    # Initialize a dictionary to store sums and counts for averaging
    sums = {
        'Total Cost': 0,
        'Capital Cost': 0,
        'Operation Cost': 0
    }
    count = 0

    # Iterate over each year for the location
    for year, data in years_data.items():
        for key in sums:
            sums[key] += data[key]
        count += 1

    # Calculate averages
    testing_averaged_results[location] = {key: value / count for key, value in sums.items()}

# Print the averaged results
print(testing_averaged_results)

{'Arizona': {'Total Cost': 32960.094, 'Capital Cost': 22717.388, 'Operation Cost': 10242.706}, 'Alaska': {'Total Cost': 42777.732, 'Capital Cost': 42777.732, 'Operation Cost': 0.0}, 'Minnesota': {'Total Cost': 181167.215, 'Capital Cost': 26604.086, 'Operation Cost': 154563.129}, 'Florida': {'Total Cost': 32698.932, 'Capital Cost': 20307.415, 'Operation Cost': 12391.517}, 'HalfMoonBay': {'Total Cost': 29901.175, 'Capital Cost': 20652.242, 'Operation Cost': 9248.933}}


In [18]:
from pathlib import Path

# Define the output file path
output_file = "Results_SA_Location_LP.xlsx"

# Store Training Result
train_df = pd.DataFrame(training_averaged_results).T 
sheet_name_train = "Training Results"

# Check if the file exists
if not Path(output_file).exists():
    # If the file does not exist, create it
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        train_df.to_excel(writer, sheet_name=sheet_name_train)
    print(f"File '{output_file}' created with sheet '{sheet_name_train}'.")
else:
    # If the file exists, append the new sheet
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        train_df.to_excel(writer, sheet_name=sheet_name_train)
    print(f"Data written to '{output_file}' in sheet '{sheet_name_train}'.")


# Store Testing Result
test_df = pd.DataFrame(testing_averaged_results).T 
sheet_name_test = "Testing Results"

# Check if the file exists
if not Path(output_file).exists():
    # If the file does not exist, create it
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        test_df.to_excel(writer, sheet_name=sheet_name_test)
    print(f"File '{output_file}' created with sheet '{sheet_name_test}'.")
else:
    # If the file exists, append the new sheet
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        test_df.to_excel(writer, sheet_name=sheet_name_test)
    print(f"Data written to '{output_file}' in sheet '{sheet_name_test}'.")

Data written to 'Results_SA_Location_LP.xlsx' in sheet 'Training Results'.
Data written to 'Results_SA_Location_LP.xlsx' in sheet 'Testing Results'.
